# Collection setup and data loading

Learn how to create Weaviate collections and import data with vectorization. We'll create the FinancialArticles collection used in other notebooks.

In [ ]:
%store -r WEAVIATE_IP
print(f'Weaviate IP: {WEAVIATE_IP}')

In [ ]:
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SESSION_TOKEN = current_credentials.token

print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SESSION_TOKEN:\t{AWS_SESSION_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

%store AWS_ACCESS_KEY
%store AWS_SECRET_KEY
%store AWS_SESSION_TOKEN

## Connect to Weaviate

Connect to a Weaviate instance using environment variables.

In [ ]:
import weaviate
import os
from weaviate.classes.init import Auth

client = weaviate.connect_to_local(
    WEAVIATE_IP,
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }        
)

client.is_ready()

## Create collection with vectorizer

Create a collection for financial articles with separate embeddings for title and content.

[Docs - Collection configuration](https://weaviate.io/developers/weaviate/manage-data/collections)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

# Delete collection if it exists
if client.collections.exists("FinancialArticles"):
    client.collections.delete("FinancialArticles")

# Create collection with named vectors for title and content
client.collections.create(
    name="FinancialArticles",

    # Configure separate embeddings for title and article content
    vector_config=[
        Configure.Vectors.text2vec_aws(
            name="title",
            source_properties=["article_title"],
            region="us-west-2",
            service="bedrock",
            model="amazon.titan-embed-text-v2:0"            
            # model="cohere.embed-multilingual-v3"
        ),
        Configure.Vectors.text2vec_aws(
            name="content",
            source_properties=["article"],
            region="us-west-2",
            service="bedrock",
            model="amazon.titan-embed-text-v2:0"
            # model="cohere.embed-multilingual-v3"
        )
    ],

    # Define property schema
    properties=[
        Property(name="article_title", data_type=DataType.TEXT),
        Property(name="article", data_type=DataType.TEXT),
        Property(name="url", data_type=DataType.TEXT),
    ],
)

## Load financial articles data

Import the financial news dataset using batch processing for efficient loading.

In [ ]:
import pandas as pd
from tqdm import tqdm
from weaviate.util import generate_uuid5

# Load the financial articles dataset
df = pd.read_parquet("data/fin_news_articles_5000.parquet")

print(f"Loaded {len(df)} articles")
print("Sample data:")
df.head()

In [ ]:
# Get collection reference
articles = client.collections.use("FinancialArticles")

# Import data with batch processing
with articles.batch.fixed_size(batch_size=100, concurrent_requests=2) as batch:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        # Create object from dataframe row
        obj = {
            "article_title": row["article_title"],
            "article": row["article"],
            "url": row["url"] if "url" in row else ""
        }

        # Generate UUID to prevent duplicates
        uuid = generate_uuid5(row["article_title"] + str(row.get("url", "")))

        batch.add_object(
            properties=obj,
            uuid=uuid
        )

        # Check for errors during import
        if batch.number_errors > 10:
            print("Too many errors during import")
            break

print(f"\nImported {len(articles)} articles")

## Check for import errors

In [ ]:
if len(articles.batch.failed_objects) > 0:
    print(f"Import completed with {len(articles.batch.failed_objects)} errors")
    for err in articles.batch.failed_objects[:5]:  # Show first 5 errors
        print(err)
else:
    print("Import completed successfully with no errors")

## Verify the data

Check that articles were imported correctly with embeddings.

In [ ]:
# Check collection size
print(f"Total articles in collection: {len(articles)}")

# Show sample data
response = articles.query.fetch_objects(limit=3)

for item in response.objects:
    print(f"Title: {item.properties['article_title']}")
    print(f"Content preview: {item.properties['article'][:100]}...\n")

In [ ]:
# Verify embeddings were created
response = articles.query.fetch_objects(
    limit=2,
    include_vector=["title", "content"]
)

for item in response.objects:
    print(f"Article: {item.properties['article_title']}")
    print(f"Title vector dimension: {len(item.vector['title'])}")
    print(f"Content vector dimension: {len(item.vector['content'])}\n")

## Test basic search

Quick test to verify the collection works with semantic search.

In [ ]:
# Test semantic search on titles
response = articles.query.near_text(
    query="technology earnings",
    target_vector="title",
    limit=3
)

print("Search results for 'technology earnings':")
for item in response.objects:
    print(f"- {item.properties['article_title']}")

## Close the client

Always close your connection when finished.

In [ ]:
client.close()